<a href="https://colab.research.google.com/github/EsbenUrbak/DisastorResponse/blob/main/etl_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# load messages dataset
messages = pd.read_csv("messages.csv")
categories = pd.read_csv("categories.csv")
df = messages.merge(categories, on='id')

In [ ]:
#cleaning of the categories column names
categories = categories["categories"].str.split(";", expand=True)
row = categories[:1]
category_names_split=row.T[0].str.split("-", expand=True)
category_colnames = category_names_split.iloc[:, [0]]
category_colnames_list = category_colnames.values.tolist()
category_colnames_list_flat = [item for sublist in category_colnames_list for item in sublist]
categories.columns = category_colnames_list_flat

In [ ]:
#Convert category values to just numbers 0 or 1.
for column in categories:
    #--First finding the position of @ in Email
    categories['pos'] = categories[column].str.find('-')
    #print(categories['pos'])
    #--Using position to slice Email using a lambda function
    categories[column] = categories.apply(lambda x: x[column][x['pos']+1:x['pos']+2],axis=1)
    #print(".")
    # set each value to be the last character of the string

    # convert column from string to numeric
    categories[column] =categories[column].astype(int) 
categories = categories.drop('pos',1)

In [ ]:
#Replace categories column in df with new category columns.
df = df.drop('categories',1)
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df.reset_index(drop=True),categories.reset_index(drop=True)], axis=1)

# drop duplicates
df = df.drop_duplicates()

In [ ]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('InsertTableName', engine, index=False)